In [1]:
def cvimage_to_pygame2(image):
    """Convert cvimage into a pygame image"""
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return pygame.image.frombuffer(image.tostring(), image.shape[1::-1],
                                   "RGB")

In [2]:
from pygame.locals import *
import pygame
import cv2
import sys
import numpy as np
import pygame_menu

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:

class Bolita(pygame.sprite.Sprite):
    def __init__(self, WIDTH, HEIGHT, velocidad):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load('bolita.png')
        self.image = pygame.transform.scale(self.image, (100,100))
        self.WIDTH = WIDTH
        self.HEIGHT = HEIGHT
        self.rect = self.image.get_rect()
        self.rect.centerx = WIDTH/2 
        self.rect.centery = 50
        self.speed = [velocidad, -velocidad]

    def actualizar(self, time, cabeza, score):
        self.rect.centerx += self.speed[0] * time
        self.rect.centery += self.speed[1] * time
        
        if self.rect.left <= 0 or self.rect.right >= self.WIDTH:
            self.speed[0] = -self.speed[0]
            self.rect.centerx += self.speed[0] * time
        if self.rect.top <= 0:
            self.speed[1] = -self.speed[1]
            self.rect.centery += self.speed[1] * time
            
        if self.rect.bottom >= self.HEIGHT:
            score[1] += 1

        if pygame.sprite.collide_rect(self, cabeza):
            score[0] += 1
            if self.rect.centerx <= cabeza.rect[0]:
                self.speed[0] = -self.speed[0]
            if self.rect.centery <= cabeza.rect[1]:
                self.speed[1] = -self.speed[1]
            if self.rect.centerx <= cabeza.rect[0] + cabeza.rect[2]:
                self.speed[0] = -self.speed[0]
            if self.rect.centery <= cabeza.rect[1] + cabeza.rect[3]:
                self.speed[1] = -self.speed[1]    
                
            self.rect.centerx += self.speed[0] * time
            self.rect.centery += self.speed[1] * time

class Cabeza(pygame.sprite.Sprite):
    def __init__(self, posicion, Wight, Hight):
        pygame.sprite.Sprite.__init__(self)
        self.radio = Wight
        self.rect = (posicion[0], posicion[1] , Wight, Hight )
#         self.rect.centerx = posicion[0]
#         self.rect.centery = posicion[1]
# 
    def mover(x,y):
        return

In [4]:
def texto(texto, posx, posy, color=(0,30,255)):
    fuente = pygame.font.SysFont("comicsansms", 72)
    salida = pygame.font.Font.render(fuente, texto, 1, color)
    salida_rect = salida.get_rect()
    salida_rect.centerx = posx
    salida_rect.centery = posy
    return salida, salida_rect

In [8]:
def start():
    
    WIDTH = 480
    HEIGHT = 640

    velocidad = 0.2
    score = [0,0]
    
    camara = 'http://192.168.1.51:4747/video'
    camera = cv2.VideoCapture(camara)
#     pygame.init()
#     pygame.display.set_caption("OpenCV camera stream on Pygame")
    screen = pygame.display.set_mode([480,640])
    
    faceClassif = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


    clock = pygame.time.Clock()

    try:
        
        cabeza = Cabeza((50,50), 50, 50)
        bola = Bolita(WIDTH, HEIGHT,velocidad)

        while True:

            ret, frame = camera.read()    
            screen.fill([0, 0, 0])

            frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
            frame = cv2.flip(frame, 1) 


            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = faceClassif.detectMultiScale(gray, 1.3, 5)
            
            for (x,y,w,h) in faces:
                cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
                cabeza = Cabeza((x ,y), w, h )

            cv2.imshow('frame',frame) 

            time = clock.tick(120)

            screen.blit(cvimage_to_pygame2(frame), (0, 0))

            bola.actualizar(time,cabeza,score)
            
            p_jug, p_jug_rect = texto(str(score[0]), WIDTH/4, 600)
            
            if(score[1] >= 1):         
                JuegoTerminado = pygame.image.load('bolita.png')
                JuegoTerminado = pygame.transform.scale(JuegoTerminado, (480,640))
                screen.blit(JuegoTerminado,(0,0))
            
            screen.blit(p_jug, p_jug_rect)
            screen.blit(bola.image, bola.rect)
            
            if(score[1] >= 1):         
                JuegoTerminado = pygame.image.load('bolita.png')
                JuegoTerminado = pygame.transform.scale(JuegoTerminado, (480,640))
                screen.blit(JuegoTerminado,(0,0))
                
            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    camera.release()
                    sys.exit(0)
                elif event.type == KEYDOWN:
                    if event.key == K_ESCAPE or event.key == K_q:
                        sys.exit(0)
                        camera.release()

    except (KeyboardInterrupt, SystemExit):
        pygame.quit()
        camera.release()
        cv2.destroyAllWindows()
#     except:
#         pygame.quit()
#         camera.release()
#         cv2.destroyAllWindows()
#         print("Unexpected error:", sys.exc_info()[0])

In [9]:
from pygame.locals import *
import pygame
import cv2
import sys
import numpy as np
import pygame_menu

pygame.init()
surface = pygame.display.set_mode((480, 640))


def start_the_game():
    start()
    pass

menu = pygame_menu.Menu(300, 400, 'Cabecea La Pelota',
                       theme=pygame_menu.themes.THEME_BLUE)

#menu.add_selector('Difficulty :', [('Hard', 1), ('Easy', 2)], onchange=set_difficulty)
menu.add_button('Play', start_the_game)
menu.add_button('Quit', pygame_menu.events.EXIT)

menu.mainloop(surface)

error: video system not initialized